In [1]:
import pandas as pd
import numpy
import os
from sentence_transformers import SentenceTransformer

import chromadb
from chromadb.utils import embedding_functions

import chromadb

import ollama


import sys
sys.path.append(os.getcwd())
from retrieval_app.retrieval.core import (
    initialize_chromadb,
    query_documents,
    get_available_collections,
    load_example_questions,
    query_seance,
    query_documents_filtered,
    query_documents_regex_filtering,
    query_documents_reranking
)
from retrieval_app.ollama_utils import (
    get_available_models,
    get_ollama_response,
    simple
)
from retrieval_app.config import BASE_DIR, DATA_DIR, DEFAULT_QUERY, DEFAULT_COLLECTION, DEFAULT_EMBEDDING_MODEL, EMBEDDINGS_DIR, EXAMPLE_QUESTIONS_FILE, CORPUS_DIR



BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR,'data')
path_to_embeddings = os.path.join(DATA_DIR,"corpus_splitted_cs1")

client = chromadb.PersistentClient(path = os.path.join(DATA_DIR,"embeddings_cs1"))
st_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="Alibaba-NLP/gte-multilingual-base",trust_remote_code=True,device="cuda",normalize_embeddings=True)

collections_list = client.list_collections()
collections_list

collection = client.get_collection(name=collections_list[0],embedding_function=st_ef)


/lre/home/apellet/anaconda3/envs/debattre-generate-questions/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-12 15:31:45.006 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequ

## retriever

In [6]:
db.get()["documents"]

['.',
 'CHAMBRE DES DÉPUTÉS Session ordinaire de 1881',
 "COIPTE RENDU IN EXTENSO. - 28 SÉANCE Séance du 20 janvier 1881 SOMMAIRE Incident : M. Cuneo d'Ornano.\n\nCommunication, par M. le président, d'une lettre par laquelle M. le marquisleValfons décline toute candidature aux fonctions de secrétaire.\n\nExcuses et demandes de congés.\n\nAllocution de M. le président d âge.\n\nTirage au sort des scrutateurs et scrutin pour la nomination du président définitif, Scrutin pour la nomination des quatre vice-présidents.\n\nScrutin pour la nomination des huit secrétaires.\n\nScrutin pour l'élection des trois questeurs. Annulatioa pour défaut de nombre et renvoi àdemain.\n\nDeuxième tour de scrutin pour l'élection d'un quatrième vice-président. - Annulation pour défaut de nombre et renvoi à demain.",
 "PBÉSIDBNCB DE M. DESSBAUX, DOYEN D'AGE.\n\nLa séance est ouverte à deux heures dixminuttS.\n\nMDreyfas, l'un des secrétaires provisoires, donne lecture du procès-verbal de la séance du11janvier.

In [21]:
queries = ["Selon le texte, combien de colons étaient concernés par les indemnités réclamées ?"]
docs = db.query(query_texts=queries)

In [22]:
docs["documents"]

[["Toutes les autres objections présentées par M. l'ordonnateur nous paraissent avoir été résolues pertinemment dans la discussion la 22 novembre 1819, et s'il pouvait en rester quelque chose, ce serait l'imprelsion d'un excès, bien plus que d'un défaut de sévérité de la part de la commission d'enquête,d.mal'admission et la vérification des mémoires de dommages fournis par les colons. Mais, quoi qu'il en loit da ces contradictions, le caractère plu.\n\ntôt parcimonieux des évaluations de la souscommission est attesté finalement par l'opinion suprême du gouverneur de la co'onie, puisque, le 2 novembre 1879, M. le commandant Olry écrivait au ministre de la marine :cCe long travail a é.6 fait avec le soin le plus consciencieux, sous la direction de M. l'ordonnateur, par la commission composée en grande partie d'habitants. - les mémoires produits par les victimes de insurrection.\n\nont été, après l'examen le plus attentif et le plus sévère, ramenés au chiff e de 908,432 fr.\n\n55 c.Ilest 

## rag

In [2]:
collection = client.get_collection(name=collections_list[0],embedding_function=st_ef)

In [3]:
def rag_query(query, n_results=3, model="llama2", context_size=4096, temperature=0):
    """Pipeline RAG : récupère depuis ChromaDB et génère une réponse avec Ollama."""
    # Étape 1 : Récupérer les documents pertinents depuis ChromaDB
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    
    # Étape 2 : Formater le contexte à partir des documents récupérés
    context = ""
    for i, doc in enumerate(results["documents"][0]):
        context += f"Document {i+1}:\n{doc}\n\n"
    
    # Étape 3 : Créer le prompt système avec le contexte
    system_prompt = f"""Tu es un assistant utile qui répond aux questions en te basant sur le contexte fourni.
    Si la réponse ne se trouve pas dans le contexte, réponds "Je n'ai pas assez d'informations pour répondre à cette question."
    
    Contexte:
    {context}
    """
    
    # Étape 4 : Créer le prompt utilisateur avec la question
    user_prompt = f"Question: {query}"
    
    # Étape 5 : Configurer les options d'Ollama
    opt = ollama.Options(num_ctx=context_size, temperature=temperature)
    
    # Étape 6 : Envoyer le prompt au modèle et obtenir la réponse
    response = ollama.chat(
        model=model,
        options=opt,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return {
        "query": query,
        "context_docs": results["documents"][0],
        "answer": response["message"]["content"]
    }


In [28]:
query = "Selon le texte, combien de colons étaient concernés par les indemnités réclamées ?"
result = rag_query(
    query, 
    n_results=3, 
    model="gemma3:27b", 
    context_size=20000, 
    temperature=0
)
print("Question:", result["query"])
print("\nRéponse:", result["answer"])

Question: Selon le texte, combien de colons étaient concernés par les indemnités réclamées ?

Réponse: Selon le texte, le total des indemnités réclamées se montait à 2 450,810 fr. 73, à répartir entre **quatre-vingt-douze** colons.


In [29]:
query = "Qui est le président de la séance ?"
result = rag_query(
    query, 
    n_results=3, 
    model="gemma3:27b", 
    context_size=20000, 
    temperature=0
)
print("Question:", result["query"])
print("\nRéponse:", result["answer"])

Question: Qui est le président de la séance ?

Réponse: D'après le Document 2, le président de la séance est M. Dessaux, doyen d'âge.


## test


In [2]:
query = "'Qui est le président de la séance ?"
system_prompt = "Tu es un assistant utile qui répond aux questions en te basant sur le contexte fourni. Si la réponse ne se trouve pas dans le contexte, réponds : Je n'ai pas assez d'informations pour répondre à cette question."

In [3]:
def query_documents(query, n_results=10):
    """Query documents from ChromaDB."""
    try:
        results = collection.query(
            query_texts=query,
            n_results=n_results
        )
        return results["ids"][0], results["documents"][0]
    except Exception as e:
        raise Exception(f"Error querying documents: {str(e)}")
    
identifiants , docs = query_documents(query,3)
context = "\n\n".join([f"Document {i+1}:\n{doc}" for i, doc in enumerate(docs)])

rag_messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Context:\n{context}\n\nQuery: {query}\n\n"}
                ]
rag_messages

[{'role': 'system',
  'content': "Tu es un assistant utile qui répond aux questions en te basant sur le contexte fourni. Si la réponse ne se trouve pas dans le contexte, réponds : Je n'ai pas assez d'informations pour répondre à cette question."},
 {'role': 'user',
  'content': "Context:\nDocument 1:\nALLOCUTION DE M. LE PRÉSIDENT D'AGE\n\nM. le président. Messieurs, je ne puis terminer la mission que le règlement m'a confiée.\n\nVoix à droite. Elle n'est pas terminée. M. le président.et laisser passer le commencement de notre dernière session sans vous remercier des marques de bienveillance dont vous avez honoré votre doyen d'âge. Ce sera pour moi un souvenir précieux ; je vous en exprime toute ma reconnaissance.\n\nQuel que soit le jugement que l'on porte sur les travaux de cette assemblée, on ne pourra contester ni son dévouement aux principes d'une sage et libérale démocratie, ni sa modération dans l'application de ces principes.\n\n(Réclamations à droite. — Non ! non ! à gauche.) 

In [7]:
response = get_ollama_response(
                    model="gemma3:27b",
                    messages=rag_messages,
                    system=system_prompt,
                    temperature=0
                )
print(response)

D'après le Document 2, le président de la séance est M. Dessaux, doyen d'âge.


## renvoi de source

In [51]:
from retrieval_app.config import SYSTEM_PROMPT_SOURCE 
query = "Selon le texte, qui a été nommé commissaire à l'enquête par les pouvoirs de la colonie?"

def query_documents(query, n_results=10):
    """Query documents from ChromaDB."""
    try:
        results = collection.query(
            query_texts=query,
            n_results=n_results
        )
        return results["ids"][0], results["documents"][0]
    except Exception as e:
        raise Exception(f"Error querying documents: {str(e)}")
identifiants , docs = query_documents(query,3)

In [37]:
SYSTEM_PROMPT = """Tu es un expert en extraction précise d'informations à partir de documents. Ta tâche principale est de localiser avec une précision absolue la source exacte d'une réponse dans un ensemble de documents.

Règles cruciales :
1. Tu dois TOUJOURS renvoyer un dictionnaire Python
2. Le dictionnaire DOIT contenir exactement deux clés :
   - `document_id`: L'identifiant unique du document source
   - `texte_source`: Le texte source EXACT sans aucune modification, correction ou reformulation
3. Si aucune réponse n'est trouvée, les valeurs seront `None`
4. Le texte source doit être copié mot pour mot depuis le document original
5. la source renvoyée doit contenir tout le contexte nécessaire pour répondre à la question"""

In [38]:
def generer_prompt_utilisateur_local(identifiants,documents, query):

    documents_numerotes = [f"Document {i}:\n{doc}" for i, doc in zip(identifiants,documents)]
    
    prompt_utilisateur = f"""Voici les documents à analyser :
{chr(10).join(documents_numerotes)}

Question à résoudre : {query}

Réponds UNIQUEMENT sous forme de dictionnaire Python en respectant strictement les règles suivantes :
- Identifie le document source de la réponse
- Copie le texte source mot pour mot
- Ne modifie JAMAIS le texte original
- La source renvoyée doit contenir suffisament de contexte pour pouvoir répondre à la question
- Retourne un dictionnaire avec `document_id` et `texte_source`

Exemple de format de réponse attendu :
{{
    "id du document": 2,
    "texte_source": "Texte exact copié du document source"
}}
"""
    
    return prompt_utilisateur

print(generer_prompt_utilisateur_local(identifiants,docs,"hello"))

Voici les documents à analyser :
Document 1881-01-20_008.txt:
Toutes les autres objections présentées par M. l'ordonnateur nous paraissent avoir été résolues pertinemment dans la discussion la 22 novembre 1819, et s'il pouvait en rester quelque chose, ce serait l'imprelsion d'un excès, bien plus que d'un défaut de sévérité de la part de la commission d'enquête,d.mal'admission et la vérification des mémoires de dommages fournis par les colons. Mais, quoi qu'il en loit da ces contradictions, le caractère plu.

tôt parcimonieux des évaluations de la souscommission est attesté finalement par l'opinion suprême du gouverneur de la co'onie, puisque, le 2 novembre 1879, M. le commandant Olry écrivait au ministre de la marine :cCe long travail a é.6 fait avec le soin le plus consciencieux, sous la direction de M. l'ordonnateur, par la commission composée en grande partie d'habitants. - les mémoires produits par les victimes de insurrection.

ont été, après l'examen le plus attentif et le plus s

In [52]:
def extract_document_data(input_string):
    """
    Extract data from a string containing a Python dictionary-like representation.
    
    Args:
        input_string (str): Input string containing a Python dictionary.
    
    Returns:
        dict: A dictionary containing document_id and texte_source
    """
    try:
        # Remove code block markers and leading/trailing whitespace
        clean_string = input_string.strip('`').strip()
        
        # Remove 'python' identifier if present
        if clean_string.startswith('python\n'):
            clean_string = clean_string[7:]
        
        # Use ast.literal_eval to safely parse the dictionary
        parsed_dict = ast.literal_eval(clean_string.strip())
        
        # Ensure the result is a dictionary
        if not isinstance(parsed_dict, dict):
            raise ValueError("Parsed content is not a dictionary")
        
        # Extract document_id and texte_source
        document_id = parsed_dict.get('document_id')
        texte_source = parsed_dict.get('texte_source')
        
        # If both are None, return the full dictionary
        if document_id is None and texte_source is None:
            return parsed_dict
        
        # Return a dictionary with the extracted values
        return {
            'document_id': document_id,
            'texte_source': texte_source
        }
    
    except (SyntaxError, ValueError, TypeError) as e:
        # If parsing fails, return a dictionary with input as both document_id and texte_source
        return {
            'document_id': input_string,
            'texte_source': input_string
        }

In [50]:
source_messages = [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": generer_prompt_utilisateur_local(identifiants,docs,query)}
                ]
source = get_ollama_response(model="gemma3:27b",messages=source_messages,
                                             system=SYSTEM_PROMPT_SOURCE,
                                             temperature=0)
source

'```python\n{\n   "document_id": "1881-01-20_009.txt",\n   "texte_source": "M. Canque, curateur aux successions vacantes, agissant, non pas au nom des membres de la famille, tous disparus, mais dans l\'intérêt. des créanciers et jusqu\'à concurrence d\'une somme de 101 308 fr. à eux dur, a pr.é la commission d\'ei quête de vouloir bipn estimer la perte subie par la succession BMïot.\\n\\nMais « la commission a été unanime pour reEousser le mémoire présenté de ce chtf par L. Canque»;dies\'est déclarée < chargée d\'apprécier les pertes subies par M. Boizot, mais non celles subies par ses créanciers.\\n\\nUne pareille décision nous semble au moins très-discutable, et, pour notre part, nous ne la saurions approuver. S\'il est un axiome de droit incontesté, c\'est que le créancier peut exercer tous les droitsdeton débiteur, et le représenter dans toute action en recouvre.\\n\\nment, restitutions et réparations de toutes sortes, et des lors il est difficile de comprendre&quel titre ce qui se

In [62]:
parsed_source = extract_document_data(source)
parsed_source

{'document_id': '1881-01-20_009.txt',
 'texte_source': "M. Canque, curateur aux successions vacantes, agissant, non pas au nom des membres de la famille, tous disparus, mais dans l'intérêt. des créanciers et jusqu'à concurrence d'une somme de 101 308 fr. à eux dur, a pr.é la commission d'ei quête de vouloir bipn estimer la perte subie par la succession BMïot.\n\nMais « la commission a été unanime pour reEousser le mémoire présenté de ce chtf par L. Canque»;dies'est déclarée < chargée d'apprécier les pertes subies par M. Boizot, mais non celles subies par ses créanciers.\n\nUne pareille décision nous semble au moins très-discutable, et, pour notre part, nous ne la saurions approuver. S'il est un axiome de droit incontesté, c'est que le créancier peut exercer tous les droitsdeton débiteur, et le représenter dans toute action en recouvre.\n\nment, restitutions et réparations de toutes sortes, et des lors il est difficile de comprendre&quel titre ce qui serait reconnu comme légitimement dû

In [63]:
id_doc , doc = query_documents(query = parsed_source["texte_source"],n_results=1)

In [66]:
doc[0]

'M. Canque, curateur aux successions vacantes, agissant, non pas au nom des membres de la famille, tous disparus, mais dans l\'intérêt. des créanciers et jusqu\'à concurrence d\'une somme de 101 308 fr. à eux dur, a pr.é la commission d\'ei quête de vouloir bipn estimer la perte subie par la succession BMïot.\n\nMais « la commission a été unanime pour reEousser le mémoire présenté de ce chtf par L. Canque»;dies\'est déclarée < chargée d\'apprécier les pertes subies par M. Boizot, mais non celles subies par ses créanciers.\n\nUne pareille décision nous semble au moins très-discutable, et, pour notre part, nous ne la saurions approuver. S\'il est un axiome de droit incontesté, c\'est que le créancier peut exercer tous les droitsdeton débiteur, et le représenter dans toute action en recouvre.\n\nment, restitutions et réparations de toutes sortes, et des lors il est difficile de comprendre&quel titre ce qui serait reconnu comme légitimement dù, à un titre quelconque, à un citoyen, pourrait

In [59]:
query_documents(query = parsed_string["texte_source"],n_results=1)[0]

['1881-01-20_009.txt']

In [33]:
parsed_string = extract_data_from_string(source)
parsed_string

{'document_id': '1881-01-20_001.txt',
 'texte_source': 'CHAMBRE DES DÉPUTÉS Session ordinaire de 1881'}

In [29]:
print_document_id("df")

df


In [35]:
def print_document_id(result):
    """
    Print the document_id if it exists in the dictionary, 
    otherwise print the full input string.
    
    Args:
        result (dict or str): Result from extract_data_from_string
    """
    if isinstance(result, dict):
        # If result is a dictionary, try to print document_id
        document_id = result.get('document_id')
        if document_id is not None :
            return document_id
        else :
            return result
        print(document_id if document_id is not None else result)
    else:
        # If result is not a dictionary, print the full input string
        return result

print_document_id(parsed_string)

'1881-01-20_001.txt'

In [ ]:
parsed_source =extract_data_from_string
                parsed_source_id = print_document_id(parsed_source)